<a href="https://colab.research.google.com/github/hemialisaaas/Fake-News-Detection/blob/main/Preprocessing_Fake_Indonews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Library**

In [ ]:
import pandas as pd
import re
import requests
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Cleaning UNWANTED PHRASES**

In [ ]:
# Load data with encoding that can handle special characters
file_path = '/content/drive/MyDrive/Fake News Detection/dataset_50k_FIX.csv'  # Sesuaikan path dengan lokasi file Anda di Google Colab
data = pd.read_csv(file_path, encoding='ISO-8859-1')

# Drop rows where content or headline are empty
data.dropna(subset=['Content', 'Headline'], inplace=True)

# Remove unwanted phrases from 'content' column
unwanted_phrases = ["ADVERTISMENT", "SCROLL TO CONTINUE WITH CONTENT"]
for phrase in unwanted_phrases:
    data['Content'] = data['Content'].str.replace(phrase, "", regex=False)

# Remove rows with empty content again after cleaning
data = data[data['Content'].str.strip() != ""]

# Reset index after dropping rows
data.reset_index(drop=True, inplace=True)

# Display first few rows to confirm
data.head()

,Label,Headline,Content
0,Real,Ridwan Kamil-Suswono Batal Gugat Hasil Pilkada...,"JAKARTA, KOMPAS.com - Ketua Tim Sukses (Timses..."
1,Real,Kubu Tri-Harris Optimistis MK Tolak Gugatan He...,"BEKASI, KOMPAS.com - Kubu pasangan calon wali ..."
2,Real,Kubu Ridwan Kamil dan Dharma Pongrekun Menyera...,"JAKARTA, KOMPAS.com - Kubu pasangan calon (pas..."
3,Real,"""Deepfake"" dan Isu SARA dalam Pilkada 2024\n\n...",HOAKS dan isu SARA sempat muncul dalam Pilkada...
4,Real,"Partisipasi Rendah, Pilkada Langsung Harus Diu...",PARTISIPASI pemilih yang relatif rendah dalam ...


In [ ]:
# Melihat total data
total_data = data.shape[0]
print(f"Total data: {total_data}")

Total data: 50356


#**DATA CLEAN & CASE FOLDING**

In [ ]:
# Mengisi NaN dengan string kosong dan memastikan semua nilai adalah string
data['Content'] = data['Content'].fillna('').astype(str)

# Fungsi pembersihan teks yang diperbarui
def clean_text(text):
    text = str(text)  # Pastikan input adalah string
    text = re.sub(r'http\S+', '', text)  # Hapus URL
    text = re.sub(r'\\x[0-9a-fA-F]{2}', '', text)  # Hapus karakter heksadesimal
    text = re.sub(r'\s+', ' ', text)  # Hapus whitespace/newline ekstra
    text = re.sub(r'RT\s+', '', text)  # Hapus 'RT' dari Retweet
    text = re.sub(r'@\w+[\w\'-]*', '', text)  # Hapus mention beserta teksnya
    text = re.sub(r'#\w+[\w\'-]*', '', text)  # Hapus hashtag beserta teksnya
    text = re.sub(r'\b[a-zA-Z]\b', '', text)  # Hapus karakter tunggal
    text = re.sub(r'\b\d+\.\d+\b', '', text)  # Hapus angka desimal (seperti jam 20.00)
    text = re.sub(r'\d+', '', text)  # Hapus angka
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Hapus kata yang mengandung angka
    text = re.sub(r'\$.{4}', '', text)  # Hapus simbol saham
    text = re.sub(r'[^\w\s]', ' ', text)  # Hapus tanda baca
    text = re.sub(r'(.)\1+', r'\1', text)  # Hapus karakter yang berulang
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Hapus karakter non-ASCII
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)  # Pisahkan dua kata yang tergabung berdasarkan perubahan huruf kecil ke besar

    return text.strip().lower()

# Terapkan fungsi pembersihan ke kolom content
data['content_clean'] = data['Content'].apply(clean_text)

# Hapus entri yang kosong setelah pembersihan
data = data[data['content_clean'].str.strip() != ""]

# Reset index setelah pembersihan
data.reset_index(drop=True, inplace=True)

# Tampilkan hasil untuk memastikan pembersihan berhasil
data.head()

,Label,Headline,Content,content_clean
0,Real,Ridwan Kamil-Suswono Batal Gugat Hasil Pilkada...,"JAKARTA, KOMPAS.com - Ketua Tim Sukses (Timses...",jakarta kompas com ketua tim sukses timses pas...
1,Real,Kubu Tri-Harris Optimistis MK Tolak Gugatan He...,"BEKASI, KOMPAS.com - Kubu pasangan calon wali ...",bekasi kompas com kubu pasangan calon wali kot...
2,Real,Kubu Ridwan Kamil dan Dharma Pongrekun Menyera...,"JAKARTA, KOMPAS.com - Kubu pasangan calon (pas...",jakarta kompas com kubu pasangan calon paslon ...
3,Real,"""Deepfake"" dan Isu SARA dalam Pilkada 2024\n\n...",HOAKS dan isu SARA sempat muncul dalam Pilkada...,hoaks dan isu sara sempat muncul dalam pilkada...
4,Real,"Partisipasi Rendah, Pilkada Langsung Harus Diu...",PARTISIPASI pemilih yang relatif rendah dalam ...,partisipasi pemilih yang relatif rendah dalam ...


#**TOKENIZING & STOPWORD**

In [ ]:
# Import libraries yang diperlukan
import pandas as pd
import re
import requests
from tqdm import tqdm

# Fungsi untuk mengunduh stopwords dari URL dan menambahkannya ke set stop_words
def download_stopwords(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Memeriksa jika ada error saat request
        return set(response.text.splitlines())
    except requests.RequestException as e:
        print(f"Error downloading stopwords from {url}: {e}")
        return set()

# Daftar URL stopwords
stopwords_urls = [
    "https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt",
    "https://raw.githubusercontent.com/nolimitid/nolimit-kamus/master/indonesian-stopwords-complete.txt",
    "https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt",
    "https://raw.githubusercontent.com/irfandythalib/python-indonesia-stopwords-remover/main/tala-stopwords-indonesia.txt"
]

# Menggabungkan stopwords dari semua URL
stop_words = set()
for url in stopwords_urls:
    stop_words.update(download_stopwords(url))

# Fungsi sederhana untuk tokenisasi menggunakan split()
def simple_tokenize(text):
    return text.split()

# Casefolding dan tokenizing
tqdm.pandas(desc="Casefolding text")
data['casefolding_content'] = data['content_clean'].progress_apply(lambda x: x.casefold())

tqdm.pandas(desc="Tokenizing text (simple split)")
data['tokenized_content'] = data['casefolding_content'].progress_apply(simple_tokenize)

# Menghapus stopwords
tqdm.pandas(desc="Removing stopwords")
data['stopwords_content'] = data['tokenized_content'].progress_apply(lambda tokens: [word for word in tokens if word not in stop_words])

# Tampilkan hasil akhir
data[['Content', 'casefolding_content', 'tokenized_content', 'stopwords_content']].head()



Removing stopwords: 100%|██████████| 50355/50355 [00:02<00:00, 20555.24it/s]


,Content,casefolding_content,tokenized_content,stopwords_content
0,"JAKARTA, KOMPAS.com - Ketua Tim Sukses (Timses...",jakarta kompas com ketua tim sukses timses pas...,"[jakarta, kompas, com, ketua, tim, sukses, tim...","[jakarta, kompas, com, ketua, tim, sukses, tim..."
1,"BEKASI, KOMPAS.com - Kubu pasangan calon wali ...",bekasi kompas com kubu pasangan calon wali kot...,"[bekasi, kompas, com, kubu, pasangan, calon, w...","[bekasi, kompas, com, kubu, pasangan, calon, w..."
2,"JAKARTA, KOMPAS.com - Kubu pasangan calon (pas...",jakarta kompas com kubu pasangan calon paslon ...,"[jakarta, kompas, com, kubu, pasangan, calon, ...","[jakarta, kompas, com, kubu, pasangan, calon, ..."
3,HOAKS dan isu SARA sempat muncul dalam Pilkada...,hoaks dan isu sara sempat muncul dalam pilkada...,"[hoaks, dan, isu, sara, sempat, muncul, dalam,...","[hoaks, isu, sara, muncul, pilkada, maluku, ut..."
4,PARTISIPASI pemilih yang relatif rendah dalam ...,partisipasi pemilih yang relatif rendah dalam ...,"[partisipasi, pemilih, yang, relatif, rendah, ...","[partisipasi, pemilih, relatif, rendah, pilkad..."


#**STEMMING**

In [ ]:
import pandas as pd

# Fungsi untuk menghapus imbuhan (prefix dan suffix)
def remove_affixes(word):
    prefixes = ['di', 'ke', 'ter', 'ber', 'meng', 'pen']
    suffixes = ['kan', 'an', 'nya']

    # Hapus prefix
    for prefix in prefixes:
        if word.startswith(prefix):
            word = word[len(prefix):]
            break

    # Hapus suffix
    for suffix in suffixes:
        if word.endswith(suffix):
            word = word[:-len(suffix)]
            break

    return word

# Daftar kata yang ingin dihapus (kata imbuhan seperti "yang," "adalah," dll.)
custom_stopwords = {
    'yang', 'adalah', 'sehingga', 'untuk', 'dengan', 'jika', 'karena', 'pada', 'dari',
    'akan', 'oleh', 'atau', 'dan', 'di', 'ke', 'ini', 'itu', 'tersebut', 'dalam',
    'saat', 'ada', 'sudah', 'masih', 'baru', 'lebih', 'kurang', 'sebagai', 'telah',
    'lagi', 'hanya', 'semua', 'tidak', 'tanpa', 'perlu', 'namun', 'mereka', 'kita',
    'anda', 'saya', 'bahwa', 'hal', 'bisa', 'terjadi', 'kemudian', 'harus',
    'apabila', 'dimana', 'tentang', 'menurut', 'banyak', 'setelah', 'sebelum',
    'dua', 'tiga', 'empat', 'tahun', 'hari', 'minggu', 'bulan', 'jam', 'pukul',
    'berita', 'indonesia', 'terakhir', 'nasional', 'internasional', 'informasi',
    'laporan', 'kabar', 'headline', 'konten', 'utama', 'ekonomi', 'politik',
    'olahraga', 'hiburan', 'sosial', 'budaya', 'update'}

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Tokenisasi kata-kata
    words = text.split()
    # Hapus imbuhan dan stopwords kustom
    cleaned_words = [remove_affixes(word) for word in words if word not in custom_stopwords]
    # Gabungkan kembali menjadi kalimat
    return ' '.join(cleaned_words)

# Terapkan ke kolom 'content' jika ada
if 'Content' in data.columns:
    # Lakukan pembersihan dan stemming
    data['stemmed_content'] = data['stopwords_content'].apply(lambda text: clean_text(str(text)))

    # Simpan hasil data yang sudah dibersihkan
    output_path = '/content/drive/MyDrive/Fake News Detection/cleaned_indonews_complete_50k.csv'
    data.to_csv(output_path, index=False)

    print(f"Data yang sudah diproses dengan stemming dan penghapusan kata tertentu disimpan di: {output_path}")
else:
    print("Kolom 'content' tidak ditemukan dalam dataset.")

# Tampilkan hasil akhir
data[['Content', 'casefolding_content', 'tokenized_content', 'stopwords_content','stemmed_content']].head()


Data yang sudah diproses dengan stemming dan penghapusan kata tertentu disimpan di: /content/drive/MyDrive/Fake News Detection/cleaned_indonews_complete_50k.csv


,Content,casefolding_content,tokenized_content,stopwords_content,stemmed_content
0,"JAKARTA, KOMPAS.com - Ketua Tim Sukses (Timses...",jakarta kompas com ketua tim sukses timses pas...,"[jakarta, kompas, com, ketua, tim, sukses, tim...","[jakarta, kompas, com, ketua, tim, sukses, tim...","['jakarta', 'kompas', 'com', 'ketua', 'tim', '..."
1,"BEKASI, KOMPAS.com - Kubu pasangan calon wali ...",bekasi kompas com kubu pasangan calon wali kot...,"[bekasi, kompas, com, kubu, pasangan, calon, w...","[bekasi, kompas, com, kubu, pasangan, calon, w...","['bekasi', 'kompas', 'com', 'kubu', 'pasangan'..."
2,"JAKARTA, KOMPAS.com - Kubu pasangan calon (pas...",jakarta kompas com kubu pasangan calon paslon ...,"[jakarta, kompas, com, kubu, pasangan, calon, ...","[jakarta, kompas, com, kubu, pasangan, calon, ...","['jakarta', 'kompas', 'com', 'kubu', 'pasangan..."
3,HOAKS dan isu SARA sempat muncul dalam Pilkada...,hoaks dan isu sara sempat muncul dalam pilkada...,"[hoaks, dan, isu, sara, sempat, muncul, dalam,...","[hoaks, isu, sara, muncul, pilkada, maluku, ut...","['hoaks', 'isu', 'sara', 'muncul', 'pilkada', ..."
4,PARTISIPASI pemilih yang relatif rendah dalam ...,partisipasi pemilih yang relatif rendah dalam ...,"[partisipasi, pemilih, yang, relatif, rendah, ...","[partisipasi, pemilih, relatif, rendah, pilkad...","['partisipasi', 'pemilih', 'relatif', 'rendah'..."


## **RANDOM **

In [ ]:
import csv
import random

def acak_baris_csv(nama_file):
    """
    Fungsi untuk mengacak isi file CSV per baris dan menyimpannya
    dengan nama file baru, tanpa mengubah urutan header.

    Args:
        nama_file: Path lengkap ke file CSV yang akan diacak.
    """
    with open(nama_file, 'r') as file_csv:
        reader = csv.reader(file_csv)
        data = list(reader)

    # Pisahkan header dan data
    header = data[0]
    data = data[1:]

    # Acak urutan baris data (tanpa header)
    random.shuffle(data)

    # Gabungkan kembali header dengan data yang sudah diacak
    data = [header] + data

    # Buat nama file baru
    nama_file_baru = nama_file[:-4] + "_acak.csv"

    # Simpan data yang sudah diacak ke file baru
    with open(nama_file_baru, 'w', newline='') as file_csv_baru:
        writer = csv.writer(file_csv_baru)
        writer.writerows(data)

# Path lengkap ke file CSV di Google Drive
nama_file = '/content/drive/MyDrive/Fake News Detection/Shuffle_50k.csv'
acak_baris_csv(nama_file)